In [6]:
import pandas_utils as pu
import pandas as pd
import numpy as np
import metadata_cleanup as cleanup
import rdkit.Chem as Chem
from rdkit_mol_identifiers import split_inchikey, ensure_smiles_column
import pubchempy as pp

In [68]:
filename = r"C:\git\msn_tree_library\data\acquisition_results\20240527_2batch_all_lib_only_detected.tsv"
tmap = r"C:\git\msn_tree_library\data\acquisition_results\20240527_public_spectral_libraries_and_new_nist23_smiles_tmap_coord.tsv"

In [69]:
df = pu.read_dataframe(filename)
tmap_df = pu.read_dataframe(tmap)

C:\git\msn_tree_library\pandas_utils.py:50: DtypeWarning: Columns (2,3,5,7,8,9,10,11,12,13,14,15,16,31,32,33,34,35,37,40,44,45,46,47,49,50,51,52,53,54,55,56,57,58,59,61,66,67,70,75,76,77,78,79,80,81,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,101,102,103,104,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,183,190,191,192,194,195,196,197,198,199,200,201,202,203,204,205,207,208,209,217,219,220,221,222,228,230,231,233,237,244,248,250,258,259,260,261,262,263,264,265,266,267,268,271,273,274,283,288,289,290,291,292,293,294,296,297,298,299,300,302,304,306,307,311,313,315,317,319,320,321,322,323,324,325,326,329,331,333,334,335,336,337,342,343,346,348,350,351,353,354,355,356,357,362,363,364,365,370) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file, sep="\t")


In [70]:
tmap_df

,canonical_smiles,msnlib,massbankeu,mona,gnps,mzcloud,nist23,entries,open,commercial,available,mhfp,x,y,availability
0,CCCCCC=CCC=CCCCCCCCC(=O)NC(Cc1ccc(O)cc1)C(=O)O,False,False,False,True,False,False,1,True,False,True,"VectorUint[54297102, 36123943, 97965370, 82611...",0.117078,-0.183075,open
1,Cc1nn(C)c(NC(=O)c2c(C)oc(C)c2)c1,False,False,False,False,True,False,1,False,True,True,"VectorUint[35566514, 82521534, 68857986, 59560...",0.012864,-0.416959,commercial
2,COc1cc2c(cc1)[nH]c(C(=O)Nc1nc3ccccc3s1)c2,False,False,False,False,True,False,1,False,True,True,"VectorUint[6233941, 83190342, 63689098, 173459...",-0.021760,-0.418086,commercial
3,C(=Nc1nc2c(CCc3ccccc3-2)s1)c1cc2c(cc1)OCO2,False,False,False,False,True,False,1,False,True,True,"VectorUint[116518462, 108369585, 269859446, 20...",0.002238,-0.371438,commercial
4,c1ccc(-c2cc3ncnc(NCc4cc5c(cc4)OCO5)c3s2)cc1,False,False,False,False,True,False,1,False,True,True,"VectorUint[62337370, 83190342, 166841738, 1818...",0.240315,0.003165,commercial
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122389,Cc1c(CN(C)C(=O)c2cc3c(cc2)n(C2CCCC2)c(=O)[nH]3...,True,False,False,False,False,False,1,False,False,False,"VectorUint[28317729, 71707353, 33228623, 16870...",0.395345,-0.021826,msnlib
122390,O=C(NCc1ccccc1Cn1cncn1)Nc1c(-c2nccs2)[nH]nc1,True,False,False,False,False,False,1,False,False,False,"VectorUint[108728796, 76444608, 30280114, 1118...",-0.055549,0.354865,msnlib
122391,CC(C)n1ncc(C(=O)Nc2ccc(Cl)cc2)c1N,True,False,False,False,False,False,1,False,False,False,"VectorUint[377807350, 103248638, 145399841, 20...",0.409326,0.159050,msnlib
122392,OC1CC(Cn2cccn2)CC1Nc1nccc(-n2ccnc2)n1,True,False,False,False,False,False,1,False,False,False,"VectorUint[73046709, 30160171, 145399841, 1118...",0.154697,-0.164870,msnlib


In [71]:
df = df[["canonical_smiles", "library"]].copy()
df.drop_duplicates("canonical_smiles")

,canonical_smiles,library
0,CCN1C(=CC(C)=O)Sc2c1cc(OC)cc2,mcebio
1,COc1c2c(C(=O)C(=O)N3CCN(C(=O)c4ccccc4)CC3)cn(C...,mcebio
2,N#Cc1c(C#N)nc2-c3cc(F)c(F)cc3C(=O)c2n1,mcebio
3,CNS(=O)(=O)c1ccccc1Nc1nc(Nc2c(OC)cc(N3CCN(C)CC...,mcebio
4,O=C1C(=Cc2cccs2)CCC1=Cc1cccs1,mcebio
...,...,...
32727,COc1c(OC)cc(C2CCCN2C(=O)Nc2c(N3CCOCC3)ccc(C(F)...,enam
32728,O=C(Cn1ccccc1=O)N1Cc2c(cccn2)N2CCCC2C1,enam
32729,Cc1noc(C2CCCN2C(=O)C2CCCCCC2)c1,enam
32730,O=C(O)CC1CCN(CC(O)c2cc(Cl)ccc2)CC1,enam


In [72]:
df

,canonical_smiles,library
0,CCN1C(=CC(C)=O)Sc2c1cc(OC)cc2,mcebio
1,COc1c2c(C(=O)C(=O)N3CCN(C(=O)c4ccccc4)CC3)cn(C...,mcebio
2,N#Cc1c(C#N)nc2-c3cc(F)c(F)cc3C(=O)c2n1,mcebio
3,CNS(=O)(=O)c1ccccc1Nc1nc(Nc2c(OC)cc(N3CCN(C)CC...,mcebio
4,O=C1C(=Cc2cccs2)CCC1=Cc1cccs1,mcebio
...,...,...
32727,COc1c(OC)cc(C2CCCN2C(=O)Nc2c(N3CCOCC3)ccc(C(F)...,enam
32728,O=C(Cn1ccccc1=O)N1Cc2c(cccn2)N2CCCC2C1,enam
32729,Cc1noc(C2CCCN2C(=O)C2CCCCCC2)c1,enam
32730,O=C(O)CC1CCN(CC(O)c2cc(Cl)ccc2)CC1,enam


In [73]:
df.groupby("library").count()

,canonical_smiles
library,
enam,10032
enammol,3334
fdadrug,2267
mcebio,8289
mcescaf,4378
nihnp,3265
otavapep,1167


In [74]:
merged = pd.merge(tmap_df, df, on="canonical_smiles", how="left")
merged

,canonical_smiles,msnlib,massbankeu,mona,gnps,mzcloud,nist23,entries,open,commercial,available,mhfp,x,y,availability,library
0,CCCCCC=CCC=CCCCCCCCC(=O)NC(Cc1ccc(O)cc1)C(=O)O,False,False,False,True,False,False,1,True,False,True,"VectorUint[54297102, 36123943, 97965370, 82611...",0.117078,-0.183075,open,NaN
1,Cc1nn(C)c(NC(=O)c2c(C)oc(C)c2)c1,False,False,False,False,True,False,1,False,True,True,"VectorUint[35566514, 82521534, 68857986, 59560...",0.012864,-0.416959,commercial,NaN
2,COc1cc2c(cc1)[nH]c(C(=O)Nc1nc3ccccc3s1)c2,False,False,False,False,True,False,1,False,True,True,"VectorUint[6233941, 83190342, 63689098, 173459...",-0.021760,-0.418086,commercial,NaN
3,C(=Nc1nc2c(CCc3ccccc3-2)s1)c1cc2c(cc1)OCO2,False,False,False,False,True,False,1,False,True,True,"VectorUint[116518462, 108369585, 269859446, 20...",0.002238,-0.371438,commercial,NaN
4,c1ccc(-c2cc3ncnc(NCc4cc5c(cc4)OCO5)c3s2)cc1,False,False,False,False,True,False,1,False,True,True,"VectorUint[62337370, 83190342, 166841738, 1818...",0.240315,0.003165,commercial,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125572,Cc1c(CN(C)C(=O)c2cc3c(cc2)n(C2CCCC2)c(=O)[nH]3...,True,False,False,False,False,False,1,False,False,False,"VectorUint[28317729, 71707353, 33228623, 16870...",0.395345,-0.021826,msnlib,enam
125573,O=C(NCc1ccccc1Cn1cncn1)Nc1c(-c2nccs2)[nH]nc1,True,False,False,False,False,False,1,False,False,False,"VectorUint[108728796, 76444608, 30280114, 1118...",-0.055549,0.354865,msnlib,enam
125574,CC(C)n1ncc(C(=O)Nc2ccc(Cl)cc2)c1N,True,False,False,False,False,False,1,False,False,False,"VectorUint[377807350, 103248638, 145399841, 20...",0.409326,0.159050,msnlib,enam
125575,OC1CC(Cn2cccn2)CC1Nc1nccc(-n2ccnc2)n1,True,False,False,False,False,False,1,False,False,False,"VectorUint[73046709, 30160171, 145399841, 1118...",0.154697,-0.164870,msnlib,enam


In [75]:
for lib in merged['library'].unique():
    merged[lib] = merged['library'] == lib

In [78]:
merged

,canonical_smiles,msnlib,massbankeu,mona,gnps,mzcloud,nist23,entries,open,commercial,...,availability,library,NaN,nihnp,fdadrug,mcescaf,mcebio,enammol,otavapep,enam
0,CCCCCC=CCC=CCCCCCCCC(=O)NC(Cc1ccc(O)cc1)C(=O)O,False,False,False,True,False,False,1,True,False,...,open,NaN,False,False,False,False,False,False,False,False
1,Cc1nn(C)c(NC(=O)c2c(C)oc(C)c2)c1,False,False,False,False,True,False,1,False,True,...,commercial,NaN,False,False,False,False,False,False,False,False
2,COc1cc2c(cc1)[nH]c(C(=O)Nc1nc3ccccc3s1)c2,False,False,False,False,True,False,1,False,True,...,commercial,NaN,False,False,False,False,False,False,False,False
3,C(=Nc1nc2c(CCc3ccccc3-2)s1)c1cc2c(cc1)OCO2,False,False,False,False,True,False,1,False,True,...,commercial,NaN,False,False,False,False,False,False,False,False
4,c1ccc(-c2cc3ncnc(NCc4cc5c(cc4)OCO5)c3s2)cc1,False,False,False,False,True,False,1,False,True,...,commercial,NaN,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125572,Cc1c(CN(C)C(=O)c2cc3c(cc2)n(C2CCCC2)c(=O)[nH]3...,True,False,False,False,False,False,1,False,False,...,msnlib,enam,False,False,False,False,False,False,False,True
125573,O=C(NCc1ccccc1Cn1cncn1)Nc1c(-c2nccs2)[nH]nc1,True,False,False,False,False,False,1,False,False,...,msnlib,enam,False,False,False,False,False,False,False,True
125574,CC(C)n1ncc(C(=O)Nc2ccc(Cl)cc2)c1N,True,False,False,False,False,False,1,False,False,...,msnlib,enam,False,False,False,False,False,False,False,True
125575,OC1CC(Cn2cccn2)CC1Nc1nccc(-n2ccnc2)n1,True,False,False,False,False,False,1,False,False,...,msnlib,enam,False,False,False,False,False,False,False,True


In [8]:
merged["library"] = merged["library"].fillna("Missing")
merged

,canonical_smiles,msnlib,massbankeu,mona,gnps,mzcloud,nist23,entries,open,commercial,...,availability,library,Unnamed: 16,nihnp,fdadrug,mcescaf,mcebio,enammol,otavapep,enam
0,CCCCCC=CCC=CCCCCCCCC(=O)NC(Cc1ccc(O)cc1)C(=O)O,False,False,False,True,False,False,1,True,False,...,open,Missing,False,False,False,False,False,False,False,False
1,Cc1nn(C)c(NC(=O)c2c(C)oc(C)c2)c1,False,False,False,False,True,False,1,False,True,...,commercial,Missing,False,False,False,False,False,False,False,False
2,COc1cc2c(cc1)[nH]c(C(=O)Nc1nc3ccccc3s1)c2,False,False,False,False,True,False,1,False,True,...,commercial,Missing,False,False,False,False,False,False,False,False
3,C(=Nc1nc2c(CCc3ccccc3-2)s1)c1cc2c(cc1)OCO2,False,False,False,False,True,False,1,False,True,...,commercial,Missing,False,False,False,False,False,False,False,False
4,c1ccc(-c2cc3ncnc(NCc4cc5c(cc4)OCO5)c3s2)cc1,False,False,False,False,True,False,1,False,True,...,commercial,Missing,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125572,Cc1c(CN(C)C(=O)c2cc3c(cc2)n(C2CCCC2)c(=O)[nH]3...,True,False,False,False,False,False,1,False,False,...,msnlib,enam,False,False,False,False,False,False,False,True
125573,O=C(NCc1ccccc1Cn1cncn1)Nc1c(-c2nccs2)[nH]nc1,True,False,False,False,False,False,1,False,False,...,msnlib,enam,False,False,False,False,False,False,False,True
125574,CC(C)n1ncc(C(=O)Nc2ccc(Cl)cc2)c1N,True,False,False,False,False,False,1,False,False,...,msnlib,enam,False,False,False,False,False,False,False,True
125575,OC1CC(Cn2cccn2)CC1Nc1nccc(-n2ccnc2)n1,True,False,False,False,False,False,1,False,False,...,msnlib,enam,False,False,False,False,False,False,False,True


In [9]:
pu.save_dataframe(merged, r"C:\git\msn_tree_library\data\acquisition_results\20240527_public_spectral_libraries_and_new_nist23_smiles_msnlib_divided_tmap_coord.tsv")

2024-12-19 13:47:02,154 - Exporting to file C:\git\msn_tree_library\data\acquisition_results\20240527_public_spectral_libraries_and_new_nist23_smiles_msnlib_divided_tmap_coord.tsv


In [36]:
compounds = pp.get_compounds("alpha-Dihydroartemisinin", "name")
for compound in compounds:
  print(compound.inchikey)

2024-12-19 10:58:46,536 - Request URL: https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/name/JSON
2024-12-19 10:58:46,537 - Request data: b'name=alpha-Dihydroartemisinin'
2024-12-19 10:58:47,347 - Created Compound(11358077)


BJDCWCLMFKKGEE-KDTBHNEXSA-N


In [ ]:
compounds = pp.get_compounds("C[C@@H]1[C@]2([C@]34[C@@](O[C@](C)(OO3)CC[C@]4([C@H](C)CC2)[H])(O[C@H]1O)[H])[H]", "smiles")
for compound in compounds:
  print(compound.inchikey)

In [ ]:
compound_name = ["aspirin"
                    # "α - Dihydroartemisinin",
                ]

compounds_dicts = {}

for compound in compound_name:
  compounds = pp.get_compounds(compound_name, "name")]
  if compounds:
        compounds_dict[smiles] = compounds[0]
        
for smiles, compound in compounds_dict.items():
    print(f"SMILES: {smiles}")

In [ ]:
df[[ID]]

In [ ]:
df[df["ID"] == "T7059"]

In [ ]:
df[df["smiles"]=="C[C@H]1[C@H](O)O[C@@H]2O[C@@]3(C)CC[C@H]4[C@H](C)CC[C@@H]1[C@@]24OO3"]

In [ ]:
df["isomeric_smiles"] = None
df = df.head(6)
df

In [ ]:
pu.save_dataframe(df, r"C:\git\msn_library\data\iocb_libraries\IOCB_cmps_test.tsv")

In [ ]:
df_template = df[["compound_name", "smiles", "cas", "library_id", "plate_id", "well_location", "unique_sample_id"]].head(6)
df_template

In [ ]:
df[["input_name", "compound_name", "smiles", "monoisotopic_mass", "iocb_name", "isomeric_smiles", "unique_sample_id"]]
# df[["compound_name", "smiles", "iocb_name"]]

In [ ]:
df["split_inchikey"] = [split_inchikey(inchikey) for inchikey in df["inchikey"]]

In [ ]:
pu.save_dataframe(df, filename)

In [ ]:
df[["compound_name", "unique_sample_id", "conc"]]

In [ ]:
df["pubchem_cid"].dtype

In [ ]:
df[["compound_name", "pubchem_cid", "input_pubchem_cid", "monoisotopic_mass", "structure_source"]]

In [ ]:
df[df["unique_sample_id"] == "pluskal_mce_1D3_K17_id"]

In [ ]:
pu.save_dataframe(df, filename)

In [ ]:
df = df.iloc[5640:]

In [ ]:
df

In [ ]:
pu.save_dataframe(df, r"C:\git\msn_library\data\library\mce_library_all_cleaned_molreference_subset2.tsv")

In [ ]:
df["id"].unique()

In [ ]:
df[df["unique_sample_id"] == "pluskal_nencka_mce_5000_A3_id"][["monoisotopic_mass", "compound_name", "molecular_species"]]


In [ ]:
df[df["compound_name"] == "polysignine"][["monoisotopic_mass", "compound_name", "molecular_species", "unique_sample_id"]]


In [ ]:
positive_df["quality_explained_intensity"] = positive_df["quality_explained_intensity"].astype(
  "float")
positive_df["num peaks"] = positive_df["num peaks"].astype("int")
positive_df["quality_explained_signals"] = positive_df["quality_explained_signals"].astype("float")
mslevels = positive_df.groupby("mslevel")
fig, ax = plt.subplots()
# Plot each group as a subplot
for name, group in mslevels:
  group['quality_explained_intensity'].hist(ax=ax, label=name)

# Add a title to the figure
ax.set_title('MSLevel Purity')

# Show the figure
plt.show()
# Create a figure with a single subplot
fig, axs = plt.subplots(len(mslevels), 1, figsize=(10, 10))

# Plot the desired column for each group
for i, (name, mslevel) in enumerate(mslevels):
  mslevel['quality_explained_intensity'].hist(ax=axs[i], label=name, bins=20, width=0.02)
  # Add a title to the subplot
  axs[i].set_title(name)

# Add a title to the figure
fig.suptitle('Your Figure Title')

# Show the figure
plt.show()
file_name = "explained_intensity"

plt.figure(figsize=(13, 9))
ax = sns.histplot(data=positive_df, x="quality_explained_intensity", binwidth=0.01, kde=False)
ax.set(xlabel="Explained Intensity")
# save_fig(file_name)

In [ ]:
wells = ["A01", "A12"]

for well in wells:
  letter = well[0]
  number = int(well[1:])
  correct = f"{letter}{number}"
  print(correct)

In [ ]:
file = r"C:\git\msn_library\data\nih\nih_library_new_headers_cleaned.tsv"

In [ ]:
df = pu.read_dataframe(file)

In [ ]:
df

In [ ]:
df[df["plate_id"] == "07P"]

In [ ]:
df = df[df["plate_id"] != "07P"]

In [ ]:
df

In [ ]:
pu.save_dataframe(df, r"C:\git\msn_library\data\nih\nih_library_new_headers_cleaned_plate7_removed.tsv")

In [ ]:
smiles = "C(C1C(C(C(C(O1)O)N)O)O)O"
mol = Chem.MolFromSmiles(smiles)
smarts = Chem.MolToSmarts(mol)
smarts

In [ ]:
file = r"C:\git\msn_library\data\weizmann\WeizMassV2_cleaned.tsv"
df = pu.read_dataframe(file)

In [ ]:
df

In [ ]:
df.drop_duplicates("inchikey")

In [ ]:
lib_df[lib_df["inchikey"].duplicated(keep=False)][["inchikey", "split_inchikey", "none"]]


In [ ]:
merged = pd.merge(df, lib_df, on="split_inchikey", how="left")
merged = merged.drop_duplicates(["split_inchikey"])
merged

In [ ]:
merged[merged["split_inchikey"].duplicated(keep=False)][["split_inchikey", "inchikey", "compound_name", "monoisotopic_mass"]]

In [ ]:
pu.save_dataframe(merged, r"C:\git\msn_library\data\weizmann\weizmass_new_split_inchikey_complete_cleaned.tsv")

In [ ]:
filtered = merged[["split_inchikey", "inchikey", "smiles", "inchi", "compound_name", "monoisotopic_mass", "id"]]
filtered

In [ ]:
pu.save_dataframe(filtered, r"C:\git\msn_library\data\weizmann\weizmass_new_split_inchikey_complete_cleaned_column_filtered.tsv")

In [ ]:
pu.read_dataframe(r"C:\git\msn_library\data\weizmann\match_NIHids_to_WizTestSet_pos_complete_table.tsv")

In [ ]:
file = r"C:\git\msn_library\data\acquisition_results\all_lib_only_detected.tsv"
df = pu.read_dataframe(file)

In [ ]:
df

In [ ]:
df_filter = df[["compound_name", "inchikey", "smiles", "inchi"]].drop_duplicates("inchikey")

In [ ]:
df_filter

In [ ]:
pu.save_dataframe(df_filter, r"C:\git\msn_library\data\acquisition_results\structures_spectral_library_19032024.tsv")

In [ ]:
df = pu.read_dataframe(r"C:\git\msn_library\data\nih\nih_library_new_headers_cleaned_plate7_removed.tsv")

In [ ]:
df["ncgc_id"]

In [ ]:
df

In [ ]:
file = r"C:\git\msn_library\data\acquisition_results\20240527_2batch_all_lib_missing.tsv"


In [ ]:
df = pu.read_dataframe(file)

In [ ]:
df

In [ ]:
sub = df[(df["library"] == "enam") | (df["library"] == "enammol")][["inchikey", "Catalog ID", "SAMPLEIDDISPLAY", "compound_name", "detected", "library", "unique_sample_id"]]
sub

In [ ]:
pu.save_dataframe(sub, r"C:\git\msn_library\data\acquisition_results\20240527_missing_compounds_enam_enammol.tsv")

In [ ]:
df.groupby(["unique_sample_id"]).count()[["inchikey"]]

In [ ]:
df[(df["plate_id"] == 5004) & (df["well_location"] == "C5")]["unique_sample_id"]

In [ ]:
file = r"C:\git\msn_library\data\public_library\NIST23tandemLib_statistics_standardized.csv"

In [ ]:
df = pu.read_dataframe(file)
df

In [ ]:
df["Positive Mode MS2"].sum() + df["Negative Mode MS2"].sum() 

In [ ]:
df["Positive Mode MSn"].sum() + df["Negative Mode MSn"].sum() 

In [ ]:
df["Positive Mode MSn"].sum() + df["Negative Mode MSn"].sum() + df["Positive Mode MS2"].sum() + df["Negative Mode MS2"].sum() 

In [ ]:
file = r"C:\git\msn_library\data\public_library\mzCloud_04.03.2024_for Customer_standardized.tsv"
df = pu.read_dataframe(file)
df

In [ ]:
df["Spectra Positive Count"].sum() + df["Spectra Negative Count"].sum() 

In [ ]:
file = r"C:\git\msn_library\data\iocb_libraries\Veverka_group\enammol_cleaned.tsv"

In [ ]:
df = pu.read_dataframe(file)
df

In [ ]:
filename = r"C:\git\msn_library\data\DIANA\diana_plate1_test_standardized.tsv"

In [ ]:
df = pu.read_dataframe(filename)
df

In [ ]:
# Filter the DataFrame using regex
filtered_df = df[df['well_location'].str.match(r'^[A-Z]3$')]
filtered_df

In [ ]:
pu.save_dataframe(filtered_df, "C:\git\msn_library\data\DIANA\diana_plate1_test_standardized_A3_Z3.tsv")